# CS-E4600 — Programming project

## Imports

In [8]:
import pandas as pd
import numpy as np

## Read file

In [73]:
file = 'Oregon-1.txt'

## Read Edges

In [74]:
pd_edges = pd.read_csv('./graphs_processed/' + file, sep=" ", skiprows=range(0, 1), header=None)
pd_edges.columns = ["FromNodeId","ToNodeId"]
#pd_edges

## Select distinct nodes

In [75]:
node_candidates_1 = pd_edges.drop_duplicates(subset="FromNodeId")["FromNodeId"]
node_candidates_2 = pd_edges.drop_duplicates(subset="ToNodeId")["ToNodeId"]

In [76]:
pd_nodes = pd.concat([node_candidates_1, node_candidates_2])
pd_nodes = pd_nodes.drop_duplicates()
#pd_nodes

## Convert Nodes to Numpy

In [78]:
nodes = pd_nodes.values
edges = pd_edges.values
#nodes, edges
nodes.size, edges.size

(10670, 44004)